<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/KK_D3_MongoDB_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

In [1]:
from datetime import datetime
import pytz
print('Tested',datetime.now(pytz.timezone('Asia/Calcutta')))

Tested 2023-08-17 14:18:06.417535+05:30


##References
https://stackoverflow.com/questions/45142629/cant-connect-to-mongo-db-via-spark <br>
https://docs.mongodb.com/spark-connector/current/python-api/

#Install & Test LOCAL MongoDB

In [2]:
#!apt install mongodb > /dev/null
#!service mongodb start
# Updated Install Sequence for MongoDB
# https://www.mongodb.com/docs/manual/tutorial/install-mongodb-on-ubuntu/
# Note Ubuntu versio 'jammy'
!curl -fsSL https://pgp.mongodb.com/server-6.0.asc | \
   sudo gpg -o /usr/share/keyrings/mongodb-server-6.0.gpg \
   --dearmor
# -----------------------
# Note reference to jammy
!echo "deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-6.0.gpg ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-6.0.list
# -----------------------
!sudo apt-get update > /dev/null
!sudo apt-get install -y mongodb-org > /dev/null              # ignore error and warning messages
#!sudo service mongodb start --- this should work but does not
# -----------------------
# https://unix.stackexchange.com/questions/715119/system-v-init-service-cant-recongnise-mongod
!sudo mkdir -p /var/log/mongodb /var/lib/mongodb
!sudo chown `whoami` /var/lib/mongodb /var/log/mongodb/
# -----------------------
# Manually start MongoDB in background mode, to free up the terminal
!nohup mongod --config /etc/mongod.conf &
!mongod --version

deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-6.0.gpg ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0 multiverse
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 9.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
nohup: appending output to 'nohup.out'
db version v6.0.9
Build Info: {
    "version": "6.0.9",
    "gitVersion": "90c65f9cc8fc4e6664a5848230abaa9b3f3b02f7",
    "openSSLVersion": "OpenSSL 3.0.2 15 Mar 2022",
    "modules": [],
    "allocator": "tcmalloc",
    "environment": {
        "distmod": "ubuntu2204",
        "distarch": "x86_64",
        "target_arch": "x86_64"
    }
}


In [ ]:
!mongod -version

db version v3.6.8
git version: 8e540c0b6db93ce994cc548f000900bdc740f80a
OpenSSL version: OpenSSL 1.1.1f  31 Mar 2020
allocator: tcmalloc
modules: none
build environment:
    distarch: x86_64
    target_arch: x86_64


In [3]:
!pip3 -qq install pymongo
from pymongo import MongoClient
import pandas as pd
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.6/603.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 7.9 MB/s eta 0:00:00


In [4]:
mongoURI = "mongodb://localhost:27017/"
client = MongoClient(mongoURI)
client.list_database_names() # ['admin', 'local']

['admin', 'config', 'local']

In [5]:
db = client.local
#db = client.sparkDB
#db = client.people
db.list_collection_names()

['startup_log']

In [6]:
table = db.startup_log
#table = db.coll01
#table = db.contacts
#table.count_documents({}) #gives the number of documents in the table
#table.find_one()

{'_id': '0468846f0b49-1692262164194',
 'hostname': '0468846f0b49',
 'startTime': datetime.datetime(2023, 8, 17, 8, 49, 24),
 'startTimeLocal': 'Thu Aug 17 08:49:24.194',
 'cmdLine': {'config': '/etc/mongod.conf',
  'net': {'bindIp': '127.0.0.1', 'port': 27017},
  'processManagement': {'timeZoneInfo': '/usr/share/zoneinfo'},
  'storage': {'dbPath': '/var/lib/mongodb'},
  'systemLog': {'destination': 'file',
   'logAppend': True,
   'path': '/var/log/mongodb/mongod.log'}},
 'pid': 1591,
 'buildinfo': {'version': '6.0.9',
  'gitVersion': '90c65f9cc8fc4e6664a5848230abaa9b3f3b02f7',
  'modules': [],
  'allocator': 'tcmalloc',
  'javascriptEngine': 'mozjs',
  'sysInfo': 'deprecated',
  'versionArray': [6, 0, 9, 0],
  'openssl': {'running': 'OpenSSL 3.0.2 15 Mar 2022',
   'compiled': 'OpenSSL 3.0.2 15 Mar 2022'},
  'buildEnvironment': {'distmod': 'ubuntu2204',
   'distarch': 'x86_64',
   'cc': '/opt/mongodbtoolchain/v3/bin/gcc: gcc (GCC) 8.5.0',
   'ccflags': '-Werror -include mongo/platform/

#Connect to and Test Remote MongoDB

#Install Spark

## Deprecated - dont use these commands

In [ ]:
!apt-get update > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

#!wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!wget -q http://apache.osuosl.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
#
# if the current version of Spark is not used, there may be errors
# check here for current versions http://apache.osuosl.org/spark
#

#!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
#!pip install -q findspark              --- no more required,  replaced by pyspark
!pip install -q pyspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

#os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 55.3 MB/s 


## Simple PIP3 install

In [7]:
!pip3 install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


#Spark2Mongo

In [8]:
#import findspark
#findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/sparkDB.coll01") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/sparkDB.coll01") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1')\
    .getOrCreate()

In [ ]:
#sc.stop()

In [9]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

##Write to MongoDB

In [10]:
people = spark.createDataFrame([("Bilbo Baggins",  50), ("Gandalf", 1000), ("Thorin", 195), ("Balin", 178), ("Kili", 77),
   ("Dwalin", 169), ("Oin", 167), ("Gloin", 158), ("Fili", 82), ("Bombur", None)], ["name", "age"])

In [11]:
people.show()

+-------------+----+
|         name| age|
+-------------+----+
|Bilbo Baggins|  50|
|      Gandalf|1000|
|       Thorin| 195|
|        Balin| 178|
|         Kili|  77|
|       Dwalin| 169|
|          Oin| 167|
|        Gloin| 158|
|         Fili|  82|
|       Bombur|null|
+-------------+----+



In [12]:
# this will send the data to the default database / collection specified during startup
people.write.format("mongo").mode("append").save()

In [13]:
# this will send the data to a new database / collection specified in this command
people.write.format("mongo").mode("append").option("database","people").option("collection", "contacts").save()

In [14]:
client.list_database_names() # ['admin', 'local']

['admin', 'config', 'local', 'people', 'sparkDB']

##Check with MongoDB client

In [15]:
client.list_database_names() # ['admin', 'local']

['admin', 'config', 'local', 'people', 'sparkDB']

In [16]:
db = client.people
list(db.list_collections())

[{'name': 'contacts',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': Binary(b"Q\x8d\x9a'\xe9\xc7M\xa4\x80(\x90;\xab\xba\x83h", 4)},
  'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}]

In [17]:
db2 = client.sparkDB
list(db2.list_collections())

[{'name': 'coll01',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': Binary(b'zR\xc2\x02\xb8\xd9D\xf2\x846b\x01)\x9a\xb2\xd5', 4)},
  'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}]

In [18]:
kollection = db.contacts
c = kollection.find({},{"_id":0})
for crnt in c:
    print(crnt)

{'name': 'Bilbo Baggins', 'age': 50}
{'name': 'Gandalf', 'age': 1000}
{'name': 'Thorin', 'age': 195}
{'name': 'Balin', 'age': 178}
{'name': 'Kili', 'age': 77}
{'name': 'Dwalin', 'age': 169}
{'name': 'Oin', 'age': 167}
{'name': 'Gloin', 'age': 158}
{'name': 'Fili', 'age': 82}
{'name': 'Bombur'}


In [19]:
pandasDF = pd.DataFrame(kollection.find({},{"_id":0}))
pandasDF

,name,age
0,Bilbo Baggins,50.0
1,Gandalf,1000.0
2,Thorin,195.0
3,Balin,178.0
4,Kili,77.0
5,Dwalin,169.0
6,Oin,167.0
7,Gloin,158.0
8,Fili,82.0
9,Bombur,NaN


##Read from MongoDB

In [20]:
sparkDF = spark.read.format("mongo").option("uri","mongodb://127.0.0.1/people.contacts").load()
sparkDF.show()

+--------------------+----+-------------+
|                 _id| age|         name|
+--------------------+----+-------------+
|{64dddfe53bd5e239...|  50|Bilbo Baggins|
|{64dddfe53bd5e239...|1000|      Gandalf|
|{64dddfe53bd5e239...| 195|       Thorin|
|{64dddfe53bd5e239...| 178|        Balin|
|{64dddfe53bd5e239...|  77|         Kili|
|{64dddfe53bd5e239...| 169|       Dwalin|
|{64dddfe53bd5e239...| 167|          Oin|
|{64dddfe53bd5e239...| 158|        Gloin|
|{64dddfe53bd5e239...|  82|         Fili|
|{64dddfe53bd5e239...|null|       Bombur|
+--------------------+----+-------------+



In [21]:
from datetime import datetime
import pytz
print('signed off at  ',datetime.now(pytz.timezone('Asia/Kolkata')))

signed off at   2023-08-17 14:23:41.250485+05:30


#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)